# Fibonacci numbers
In this tutorial we will discuss how to perform the fibonacci numbers using *Python*. There is a lot of tutorials on the Internet describing various way to calculate N-th element of this sequence. We will add extra aspects to these considerations.
### What is the fibonacci sequence?
At the beginning let's just write what are the fibonacci numbers: `F(N)`. The two first elements are defined as 0 and 1, i.e. `F(0) = 0`, `F(1) = 1`. The next one is sum of the two previous elements: `F(N) = F(N-2) + F(N-1)`, so the first ten elements are equal: `0, 1, 2, 3, 5, 8, 13, 21, 43, 55`. OK, now let's define three different functions allowing to compute the N-th element of the sequence: